In [207]:
#display_read_well_csv()

# def get_well_info(filename='elisawells.csv',
#                  rows=8):
#     import csv, sys
    
#     rows = rows
#     row_count = 0
#     col_count = 0
    
#     # create list of lists
#     col_list = list()
    
#     with open(filename, 'rb') as f:
#         reader = csv.reader(f)
#         try:
#             col_count_list = list()
            
#             for row in reader:
#                 row_value = int(row[0])
                
#                 if row_count >= rows:
#                     row_count = 0
#                     col_count = col_count + 1
#                     #print col_count_list
#                     col_list.append(col_count_list)
#                     col_count_list = list()
                
#                 row_count = row_count + 1
#                 col_count_list.append(row_value)
                
#                 #print '%i\t%i\t%i' % (row_count, col_count, row_value)
            
#             if col_count_list:
#                 col_list.append(col_count_list)
            
#         except csv.Error as e:
#             # TODO handle
#             print 'file %s, line %d: %s' % (filename, reader.line_num, e)
    
#     col_resize = list()
    
#     for i in range(rows):
#         tmp = list()
        
#         for j in range(len(col_list)):
#             tmp.append(col_list[j][i])
            
#         col_resize.append(tmp)
    
#     return col_resize


# get well xy pos for num
# for i in range(len(well_map)):
#     for j in well_map[i]:
#         print j
#         well_pos = j-1
#         well_col = abs(well_pos/rows)+1
#         well_row = abs(well_pos%rows)
        
#         print well_col
#         print well_row

In [208]:
import csv, sys
import numpy as np
import pandas as pd


In [209]:
def read_plate_file_to_csv(filename):
    
    csv_list = list()
    
    def get_csv_from_row(row):
        return row.split('=')[1].split(' ')
    
    with open(filename) as f:
        content = f.readlines()
    
    for line in content:
        # -1 is not found
        if line.find('Row') > 0:
            csv_list.append(get_csv_from_row(line.strip()))
            
    return csv_list

In [210]:
# from csv to plate array
# def read_plate_csv(filename):
#     import csv, sys
    
#     csv_list = list()
#     with open(filename, 'rb') as f:
#         reader = csv.reader(f)
#         try:
#             for row in reader:
#                 csv_list.append(row)
#         except csv.Error as e:
#             # TODO handle
#             print 'file %s, line %d: %s' % (filename, reader.line_num, e)
#     print 'oo'
#     print csv_list
#     return csv_list

# from plate array to mapped plate array
def get_mapped_plate(plate, filename='elisawells.csv',
                 rows=8, cols=12):
    import csv, sys
    
    row_count = 0
    col_count = 0
    
    mapped_plate = list()
       
    with open(filename, 'rb') as f:
        reader = csv.reader(f)
        col_count_list = list()

        for row in reader:
            row_value = int(row[0])
            ab2_value = float(row[1])
    
            well_pos = row_value
            well_col = abs(well_pos/rows)
            well_row = abs(well_pos%rows)-1
            
            mapped_plate.append((row_value, ab2_value, plate[well_row][well_col]))
            
    return mapped_plate


In [211]:
def get_prot_concentration_step(p_high):
    p = p_high

    return [0, p/8.0, p/4.0, p/2.0, p]

def get_coating_ab_step(a_high):
    p = a_high

    return [p/8.0, p/4.0, p/2.0, p]

def get_well_attrs(mapped_plate, prot_concentration_high, coating_ab, ab2_step):
    prot_step = get_prot_concentration_step(prot_concentration_high)
    coating_step = get_coating_ab_step(coating_ab)

    count = 0
    well_attr = list()
    for i in prot_step:
        for j in coating_step:
            for k in range(ab2_step):
                # join both tuples
                tmp_tup = ((j,i)+(mapped_plate[count]))
                well_attr.append({'coating_ab': tmp_tup[0],
                                  'prot': tmp_tup[1],
                                  'well': tmp_tup[2],
                                  'ab2': tmp_tup[3],
                                  'value': float(tmp_tup[4]),
                                 })
                count = count + 1
    return well_attr

In [212]:
# plate = read_plate_csv('plate_1_dummy.csv') # old


In [213]:
prot_concentration_high = 2.0
coating_ab = 4.0
rows = 8
cols = 12

# ab2_step = 4
# how many ab2 steps determines coating #

# plate from file to pandas dataframe for bokeh app
def read_plate_to_df(plate_file_csv,
                    prot_concentration_high=prot_concentration_high,
                    coating_ab=coating_ab,
                    ab2_step=4):

    plate = read_plate_file_to_csv(plate_file_csv)
    mapped_plate = get_mapped_plate(plate)
    well_attr = get_well_attrs(mapped_plate,
                               prot_concentration_high,
                               coating_ab,
                               ab2_step)
    df = pd.DataFrame(well_attr)
    return df

plate_file = '161102-001.CSV'
df = read_plate_to_df(plate_file,
                     prot_concentration_high,
                     coating_ab)

In [214]:
# plot one series
#df[(df.prot == 2) & (df.coating_ab == 1)]
#df

In [215]:
#prot_step = get_prot_concentration_step(prot_concentration_high)
#    for p in prot_step:

# get plot's worth of data points
def get_df_by_prot_conc(df, prot_conc):
    return df[(df.prot == prot_conc)]

print df.prot.unique()

prot_conc_05 = get_df_by_prot_conc(df, 0.5)
prot_conc_05.ab2.unique()

# get each point (coating) data 
#get_scatter_df_by_coating
#df['value'] = df['value'].astype('float64') 

df.dtypes
#df.value

[ 0.    0.25  0.5   1.    2.  ]


ab2           float64
coating_ab    float64
prot          float64
value         float64
well            int64
dtype: object

In [216]:
ab2_steps = prot_conc_05.sort_values('ab2').ab2.unique().astype(str).tolist()

In [217]:
ab2_steps

['0.25', '0.5', '1.0', '2.0']

In [218]:
['0.25', '0.5', '1.0', '2.0']

['0.25', '0.5', '1.0', '2.0']

In [263]:
rows = 8.0
cols = 12.0


In [270]:
for i in [77]:
    print('>%i' % (i))
    row_val = int((i)/cols)
    print(row_val)
    col_val = int(i%cols)
    print(col_val)
    print('---')
    numwise = (col_val*rows)+row_val
    print(numwise)
    print('*******\n\n')
    
def get_well_num_from_table_select_num(num, rows, cols):
    row_val = int((i)/cols)
    col_val = int(i%cols)
    numwise = (col_val*rows)+row_val
    numwise = numwise+1 # human readable well
    return numwise

get_well_num_from_table_select_num(77, rows, cols)
    
#df[df.well == 47]

>77
6
5
---
46.0
*******




47.0

In [229]:
df.loc[:,'exclude'] = pd.Series(False, index=df.index)

In [240]:
exclude = [1,5,3,79]
for e in exclude:
    df.loc[e,('exclude')] = True

In [241]:
df.loc[79,]

ab2               2
coating_ab        4
prot              2
value         34590
well             47
exclude        True
Name: 79, dtype: object

In [242]:
df[(df.exclude == False)]

,ab2,coating_ab,prot,value,well,exclude
0,0.50,0.5,0.00,960.0,20,False
2,1.00,0.5,0.00,1040.0,28,False
4,1.00,1.0,0.00,900.0,5,False
6,2.00,1.0,0.00,1220.0,29,False
7,0.25,1.0,0.00,5640.0,75,False
8,2.00,2.0,0.00,1380.0,38,False
9,0.50,2.0,0.00,2640.0,67,False
10,0.25,2.0,0.00,2900.0,71,False
11,1.00,2.0,0.00,5050.0,76,False
12,1.00,4.0,0.00,900.0,6,False


In [331]:
filename = '161102-001.CSV'
plate_files = [
    '161102-001.CSV',
    '161102-002.CSV',
    '161102-003.CSV',
]

def plate_metadata(plate_file):
    
    with open(filename) as f:
        content = f.readlines()
            
    string_metadata = ''.join(content[:4]).strip()
    return string_metadata


def write_csv_metadata(plate_dir,
                       coating_ab,
                       prot,
                       ab2,
                       coating_ab_units,
                       prot_units,
                       ab2_units,
                       plate_files,
                       output_file,
                       name):
    
    # TODO random directory    

    # TODO write mean set (mean function)

    with open(output_file, 'w') as csvfile:
        fieldnames = ['filename',
                      'metadata',
                     'coating_ab',
                     'coating_ab_units',
                     'prot',
                     'prot_units',
                     'ab2',
                     'ab2_units',
                     'name']

        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        
        for plate_file in plate_files:
            # todo sort out the fact the ab/prot names are repeated
            writer.writerow({'filename': plate_file,
                             'metadata': plate_metadata(plate_file),
                             'coating_ab': coating_ab,
                             'coating_ab_units': coating_ab_units,
                             'prot': prot,
                             'prot_units': prot_units,
                             'ab2': ab2,
                             'ab2_units': ab2_units,
                             'name': name,
                            })
            


In [333]:
plate_dir = '/mnt/ceph/mbp/servers/bioinformatics-platform/home/steve/projects/elisa/elisa_results/data/example'

write_csv_metadata(plate_dir,
                   'ab_one', 'm',
                   'Mr Protein', 'ng/ml',
                   'ab_two', 'ug/ml',
                  plate_files,
                  'metadata.csv',
                  'myreport')

In [326]:
# for display on index and individually
def read_plate_file_to_csv_metadata(plate_filename=None,
                           metadata_file='metadata.csv'):
    
    csv_list = list()
    
    def get_csv_from_row(row):
        return row.split('=')[1].split(' ')
    
    d = []
    with open(output_file) as f:
        reader = csv.DictReader(f)
        for row in reader:
            if plate_filename:
                if plate_filename == \
                    row['filename']:
                        d.append(row)
                        break
            else:
                d.append(row)

    return d

metadata_file = 'metadata.csv'

read_plate_file_to_csv_metadata(plate_filename='161102-001.CSV')

[{'ab2': 'Mr Protein',
  'ab2_units': 'ug/ml',
  'coating_ab': 'ab_one',
  'coating_ab_units': 'ng/ml',
  'filename': '161102-001.CSV',
  'metadata': '[TP{001}, 02.11.2016, 14:10:59]\nMeasureMode=Sunrise_Measure\nConfiguration 1=405:\nKineticsMeasTime 1=0',
  'prot': 'm',
  'prot_units': 'ab_two'}]

In [358]:
APP_DATA = '/mnt/ceph/mbp/servers/bioinformatics-platform/home/steve/projects/elisa/elisa_results/data'

In [359]:
def read_plate_file_to_csv_metadata(plate_dir,
                           plate_filename=None,
                           metadata_filename='metadata.csv'):
    
    csv_list = list()
    metadata_file = '%s/%s' % (plate_dir, metadata_filename)
    
    def get_csv_from_row(row):
        return row.split('=')[1].split(' ')
    
    d = []
    with open(metadata_file) as f:
        reader = csv.DictReader(f)
        for row in reader:
            if plate_filename:
                if plate_filename == \
                    row['filename']:
                        d.append(row)
                        break
            else:
                d.append(row)

    return d

import os

def get_index(directory):
    
    index = []
    for x in os.walk('%s/' % directory):
        path = x[0]
        
        r = {}
        r['dir'] = path
        r['dirname'] = os.path.basename(path)
        
        # first result is always parent dir
        if r['dirname'] == '':
            continue
        
        r['metadata'] = read_plate_file_to_csv_metadata(plate_dir)        
        
        index.append(r)
    
    return index

get_index(APP_DATA)

[{'dir': '/mnt/ceph/mbp/servers/bioinformatics-platform/home/steve/projects/elisa/elisa_results/data/example',
  'dirname': 'example',
  'metadata': [{'ab2': 'Mr Protein',
    'ab2_units': 'ug/ml',
    'coating_ab': 'ab_one',
    'coating_ab_units': 'ng/ml',
    'filename': '161102-001.CSV',
    'metadata': '[TP{001}, 02.11.2016, 14:10:59]\nMeasureMode=Sunrise_Measure\nConfiguration 1=405:\nKineticsMeasTime 1=0',
    'prot': 'm',
    'prot_units': 'ab_two'},
   {'ab2': 'Mr Protein',
    'ab2_units': 'ug/ml',
    'coating_ab': 'ab_one',
    'coating_ab_units': 'ng/ml',
    'filename': '161102-002.CSV',
    'metadata': '[TP{001}, 02.11.2016, 14:10:59]\nMeasureMode=Sunrise_Measure\nConfiguration 1=405:\nKineticsMeasTime 1=0',
    'prot': 'm',
    'prot_units': 'ab_two'},
   {'ab2': 'Mr Protein',
    'ab2_units': 'ug/ml',
    'coating_ab': 'ab_one',
    'coating_ab_units': 'ng/ml',
    'filename': '161102-003.CSV',
    'metadata': '[TP{001}, 02.11.2016, 14:10:59]\nMeasureMode=Sunrise_Measu